In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/Mistral-7B-Instruct-v0.2-physics-1k-woody"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

403

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1584

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for mo

 LLaMA-2 was trained on 2 trillion tokens of data.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
 LLaMA-2 was trained on 2 trillion tokens of data.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###Mamba Questions

In [ ]:
# Mamba Questions

questions_file_path = '/content/questions/MambaQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 Mamba achieves computational efficiency
by using efficient algorithms for state expansion and attention operations, as well as by utilizing parallelism and batching techniques. It also uses
a simplified block design that combines the H3 block with the ubiquitous MLP block, replacing the first multiplicative gate with an activation
function and adding an SSM to the main branch. Additionally, Mamba uses the SiLU/Swish activation function and fixes the number of layers to 2 in all
models except Transformer++.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Mamba introduces a selection mechanism to structured
state space models, allowing them to perform context-dependent reasoning while scaling linearly in sequence length. This mechanism is then
incorporated into a simple attention-free architecture, resulting in state-of-the-art performance on a diverse set of domains. Additionally, Mamba is
shown to match or exceed the performance of strong Transformer models.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Mamba's performance evaluation
suggests several areas for future research and development. One area is improving the scalability of structured state space models (SSMs) by
addressing further engineering challenges and adjustments. Another area is exploring the potential of selective SSMs in building foundation models for
different domains, particularly those requiring long context like genomics, audio, and video. Additionally, there is room for improvement in the
selectivity mechanism used by Mamba, as well as investigating the effects of interleaving the Mamba block with other architectures. Finally,
understanding the role of selective parameters in the SSM could lead to new insights and optimizations.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/M

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Dynamic parameter adjustment plays a
crucial role in Mamba's selective state space models by enabling context-dependent reasoning while maintaining linear scalability in sequence length.
The authors propose a selection mechanism that allows the model to filter out irrelevant information and compress the context into an efficient state.
This mechanism is inspired by the adaptive nature of human perception and cognition, and it significantly improves the performance of the model across
various domains. By incorporating this mechanism into a simple attention-free architecture, Mamba achieves state-of-the-art results on a diverse set
of tasks, demonstrating its potential as a general sequence model backbone.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Mamba addresses the inefficiencies
of Transformers in moderate to long sequence processing by using a selective state space model (SSM) layer, which allows the model to selectively
remember relevant tokens while ignoring irrelevant ones. This enables Mamba to achieve perfect performance on tasks involving long sequences, whereas
other methods struggle to go beyond a factor of 2. Additionally, Mamba uses a linear time complexity algorithm, reducing the computational cost
compared to quadratic time complexity algorithms used by Transformers.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
 The design 

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How does Mamba achieve computational efficiency without specialized hardware optimizations?
 response:  Mamba achieves computational efficiency
by using efficient algorithms for state expansion and attention operations, as well as by utilizing parallelism and batching techniques. It also uses
a simplified block design that combines the H3 block with the ubiquitous MLP block, replacing the first multiplicative gate with an activation
function and adding an SSM to the main branch. Additionally, Mamba uses the SiLU/Swish activation function and fixes the number of layers to 2 in all
models except Transformer++.


Q1: What innovations does Mamba introduce to manage long sequence data processing?
 response:  Mamba introduces a selection mechanism to structured
state space models, allowing them to perform context-dependent reasoning while scaling linearly in sequence length. This mechanism is then
incorporated into a simple attention-free architecture, resulting in state-of-the-art perfo

In [ ]:
json_file_name = 'MambaQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'MambaQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Parametric Magnon Questions

In [ ]:
# Parametric Magnon Questions

questions_file_path = '/content/questions/ParametricMagnonQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The materials commonly used to
construct quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However, these
materials face fabrication challenges for wafer-scale integration. In this work, the authors present a different approach by using wafer-compatible
materials to engineer a hybrid transducer that exploits magnon nonlinearities in a ferromagnetic microdisk to mediate the microwave interaction with
spin qubits in silicon carbide.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Magnon nonlinearities can enhance quantum transduction by allowing
selective tuning of the spin-magnet coupling "on" and "off." This control over the coupling enables protection of spin centers against resonant magnon
noise-induced decoherence, leading to improved transduction behavior and tunability.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Parametric magnonics introduces a new approach
to quantum transducers by using wafer-compatible materials to engineer a hybrid transducer that exploits magnon nonlinearities in a magnetic microdisc
to address quantum spin defects in silicon carbide. This interaction scheme points to the unique transduction behavior that can be obtained when
complementing quantum systems with nonlinear magnonics.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Introducing nonlinear magnonics provides alternative
perspectives for engineering quantum interfaces to spin qubits and motivates further research into uncovering the interesting phenomena lying at the
intersection of nonlinear magnonics and quantum systems. It also offers potential for generating squeezed magnon states, which can exponentially
enhance the coupling strengths and cooperativities in hybrid quantum systems.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Magnon nonlinearities in quantum systems could
potentially lead to exponentially enhanced coupling strengths and cooperativities, which could be useful for generating squeezed magnon states and
improving the performance of hybrid quantum systems. Additionally, they could enable new types of quantum operations and functionalities beyond what
is currently achievable with linear magnon transduction. For example, they could be used to create entangled states of multiple qubits, perform
quantum error correction, and implement quantum algorithms. Overall, the integration of magnon nonlinearities into quantum systems could open up new
avenues for exploring the fundamental principles of quantum mechanics and developing novel technologies for quantum communication and computation.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
 Recent researc

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What materials are typically used to construct quantum transducers in hybrid quantum systems?
 response:  The materials commonly used to
construct quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However, these
materials face fabrication challenges for wafer-scale integration. In this work, the authors present a different approach by using wafer-compatible
materials to engineer a hybrid transducer that exploits magnon nonlinearities in a ferromagnetic microdisk to mediate the microwave interaction with
spin qubits in silicon carbide.


Q1: How do magnon nonlinearities enhance quantum transduction?
 response:  Magnon nonlinearities can enhance quantum transduction by allowing
selective tuning of the spin-magnet coupling "on" and "off." This control over the coupling enables protection of spin centers against resonant magnon
noise-induced decoherence, leading to improved transduction behavior and tunability.

In [ ]:
json_file_name = 'ParametricMagnonQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'ParametricMagnonQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Quantum Mechanics Questions

In [ ]:
# Quantum Mechanics Questions

questions_file_path = '/content/questions/QuantumMechanicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 Quantum mechanics has had a significant impact on modern
technology, leading to the development of fields such as quantum information theory and quantum thermodynamics, as well as the improvement of existing
technologies through the use of quantum principles. This has resulted in advancements in areas such as superfast quantum computers, unbreakable
quantum cryptography, and ultrasensitive quantum sensors, among others. These innovations have the potential to revolutionize various industries and
have already begun to rival the three major industrial revolutions of the last century. Additionally, research in quantum science continues to push
the boundaries of our understanding of the physical world, with implications for fields ranging from condensed matter physics to cosmology.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Quantum mechanics has had a significant influence on the development of new
fields of study, such as quantum information theory and quantum thermodynamics. It has also led to the creation of novel mathematical and
computational tools applicable to various areas, including condensed matter physics, statistical mechanics, and cosmology. Additionally, research in
quantum science has achieved tangible impacts through the improvement of understanding of the resource power of quantum phenomena, triggering a
technological overhaul that is comparable to the three major industrial revolutions of the last century. The potential applications of quantum science
include superfast quantum computers, unbreakable quantum cryptography, and ultrasensitive quantum sensors, among others.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 There are several fundamental questions about quantum
mechanics that remain open, such as the interpretation of its elusive foundations, the relationship between quantum mechanics and general relativity,
and the emergence of the classical world via quantum principles. Additionally, there are ongoing efforts to explore the interplay of quantum mechanics
with other areas of physics, such as black hole physics and thermodynamics, and to develop disruptive technologies based on quantum principles.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Exploring quantum mechanics contributes to theoretical
physics by shedding light on the physical meaning of fundamental quantum principles and pushing the boundaries of the quantum description of the
world. This effort has led to significant advancements in understanding the interplay of quantum mechanics with other areas such as black hole
physics, thermodynamics, and the emergence of the classical world. Additionally, it has resulted in the development of innovative technologies like
topological quantum computing and prospects of a quantum internet.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Decades of progress in the experimental verification and
control of quantum systems have routinely proven detractors wrong. There is little doubt that quantum mechanics is a valid description of the physical
reality.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
 Future developments in quantum technology include superfast quantum
computers, unbreakable quantum cryptography, ultrasensitive quantum sensors, topological quantum computing, and prospects of a quantum internet. These
advancements have the potential to disrupt various industries and revolutionize our understanding of the physical world.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the significance of quantum mechanics in modern technology?
 response:  Quantum mechanics has had a significant impact on modern
technology, leading to the development of fields such as quantum information theory and quantum thermodynamics, as well as the improvement of existing
technologies through the use of quantum principles. This has resulted in advancements in areas such as superfast quantum computers, unbreakable
quantum cryptography, and ultrasensitive quantum sensors, among others. These innovations have the potential to revolutionize various industries and
have already begun to rival the three major industrial revolutions of the last century. Additionally, research in quantum science continues to push
the boundaries of our understanding of the physical world, with implications for fields ranging from condensed matter physics to cosmology.


Q1: How has quantum mechanics influenced new fields of study?
 response:  Quantum mechanics has had a significant influence o

In [ ]:
json_file_name = 'QuantumMechanicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'Quantum MechanicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Qubit Teleportation Questions

In [ ]:
# Qubit Teleportation Questions

questions_file_path = '/content/questions/QubitTeleportationQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The main components used in quantum networks for
teleportation include solid-state spin qubits, entanglement swapping on the middle node, and storage in a memory qubit. Additionally, there are
innovations such as improved qubit readout procedures, active memory qubit protection during entanglement generation, and tailored heralding to reduce
remote entanglement infidelities.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Entanglement is established between distant nodes in a
quantum network through an entanglement swapping protocol mediated by a third node, such as a quantum repeater protocol. This process creates a shared
entangled state between the distant nodes, allowing for reliable quantum information transfer.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Memory qubits play a crucial role in the
process of quantum teleportation in a network. They serve as a reliable medium for storing the entangled state generated during the teleportation
protocol. This allows for the efficient transfer of quantum information between distant nodes without the need for direct physical contact.
Additionally, memory qubits enable the extension of the memory preservation time through active coherence protection from the spin bath. Overall,
memory qubits are essential components of a quantum network, enabling the implementation of advanced teleportation protocols and paving the way for
future quantum technologies.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Potential future applications of quantum
teleportation in quantum networks include executing and testing multi-node protocols and applications through platform-independent control software,
which is an important prerequisite for a large-scale future network. Additionally, improvements in phase stabilization and extending current schemes
for use in deployed fiber are expected to yield higher fidelity in teleportation.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Real-time feed-forward plays a crucial role in achieving
unconditional quantum teleportation by enabling state transfer each time a qubit state is inserted into the teleporter. The presence of feed-forward
operations significantly improves the average state fidelity compared to the case where no feed-forward is applied. However, it also introduces
additional error sources such as depolarizing noise on Alice during the decoupling sequence and ionization probability on Alice. These error sources
must be carefully considered and mitigated to ensure successful teleportation.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
 To achieve
efficient quantum teleportation between non-neighbouring nodes, several technical advancements are required. These include improvements in the
fidelity of entanglement generation, the development of

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What are the main components used in quantum networks for teleportation?
 response:  The main components used in quantum networks for
teleportation include solid-state spin qubits, entanglement swapping on the middle node, and storage in a memory qubit. Additionally, there are
innovations such as improved qubit readout procedures, active memory qubit protection during entanglement generation, and tailored heralding to reduce
remote entanglement infidelities.


Q1: How is entanglement established between distant nodes in a quantum network?
 response:  Entanglement is established between distant nodes in a
quantum network through an entanglement swapping protocol mediated by a third node, such as a quantum repeater protocol. This process creates a shared
entangled state between the distant nodes, allowing for reliable quantum information transfer.


Q2: What role does the Bell-state measurement (BSM) play in quantum teleportation?
 response:  The Bell-state measurement (BSM) plays a 

In [ ]:
json_file_name = 'QubitTeleportationQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QubitTeleportationQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Variance Based Questions

In [ ]:
# Variance Based Questions

questions_file_path = '/content/questions/VarianceQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The focus of variance-based sensitivity analysis in
quantum memory is to identify which input parameters are most sensitive globally and probe for correlations between parameters that can be leveraged
to maintain acceptable system performance at nonoptimal parameter values. It provides a complete picture of the system performance landscape around a
central point of input parameters.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Λ-type quantum memory refers to a type of quantum memory that uses a Λ-shaped energy level
scheme to store and retrieve qubits. It is often used in optical systems due to its high efficiency and long coherence time.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Memory sensitivity analysis is important because it allows for
identification of which input parameters are most sensitive to fluctuations, enabling optimization of system performance and compensation for
nonoptimal parameter values. It also probes for correlations between parameters, which can be leveraged to achieve acceptable performance even if some
parameters are not optimized. In the context of quantum memory, sensitivity analysis can be applied to various performance metrics such as efficiency,
fidelity, storage time, etc., allowing for a comprehensive understanding of the system's behavior under different conditions.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The experimental techniques used to analyze memory sensitivity
include variance-based sensitivity analysis, optimization-based analysis, and Monte Carlo simulations. These methods allow for the determination of
how sensitive a quantum memory is to fluctuations in memory parameters and experimental drift. The results of these analyses can provide valuable
insights into the robustness and reliability of different memory protocols.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Gaussian control field relates to memory performance through
its optimization for maximum efficiency in storing and retrieving information from the memory. The control field parameters, such as pulse area,
delay, and duration, define the temporal envelope of the control field and are assumed to have been optimized for maximum memory efficiency. These
parameters are grouped into intrinsic and extrinsic categories, with the latter being more readily tunable. The analysis is also partitioned based on
whether the control field defines a Gaussian temporal envelope or not. The sensitivity of the memory to variations in these control field parameters
is calculated using one-at-a-time analysis and single-parameter sensitivity calculations, revealing the impact of each parameter on the overall memory
efficiency. Additionally, the overlap fidelity between the optimal control fields at neighboring points in memory space is studied, confirming the
intuition that the region of least overlap co

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the focus of variance-based sensitivity analysis in quantum memory?
 response:  The focus of variance-based sensitivity analysis in
quantum memory is to identify which input parameters are most sensitive globally and probe for correlations between parameters that can be leveraged
to maintain acceptable system performance at nonoptimal parameter values. It provides a complete picture of the system performance landscape around a
central point of input parameters.


Q1: What does Λ-type quantum memory refer to?
 response:  Λ-type quantum memory refers to a type of quantum memory that uses a Λ-shaped energy level
scheme to store and retrieve qubits. It is often used in optical systems due to its high efficiency and long coherence time.


Q2: Why are control field parameters crucial in Λ-type quantum memory systems?
 response:  Control field parameters are crucial in Λ-type quantum
memory systems because they determine the efficiency of the memory process. Fluctuations and drift

In [ ]:
json_file_name = 'VarianceQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'VarianceQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)